### Linear and Multilinear Functions

In [117]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [118]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(1.56 𝐞₁ + 2.99 𝐞₂ + 2.8 𝐞₃ + -0.706 𝐞₄,
 1.56 𝐞₁ + 2.99 𝐞₂ + 2.8 𝐞₃ + -0.706 𝐞₄)

In [119]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(11.6 𝐞₁ + -0.0601 𝐞₂ + -4.63 𝐞₃ + 2.76 𝐞₄,
 11.6 𝐞₁ + -0.0601 𝐞₂ + -4.63 𝐞₃ + 2.76 𝐞₄)

In [120]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [121]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(-1.18 𝐞₁ + 2.37 𝐞₂ + 2.3 𝐞₃ + -0.378 𝐞₄,
 -1.18 𝐞₁ + 2.37 𝐞₂ + 2.3 𝐞₃ + -0.378 𝐞₄)

In [122]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.416 + 1.98 𝐞₁ + -0.65 𝐞₂ + 2.91 𝐞₃ + -1.04 𝐞₄ + -0.267 𝐞₁₂ + -0.781 𝐞₁₃ + 0.219 𝐞₁₄ + 0.649 𝐞₂₃ + -0.212 𝐞₂₄ + -0.0878 𝐞₃₄,
 0.416 + 1.98 𝐞₁ + -0.65 𝐞₂ + 2.91 𝐞₃ + -1.04 𝐞₄ + -0.267 𝐞₁₂ + -0.781 𝐞₁₃ + 0.219 𝐞₁₄ + 0.649 𝐞₂₃ + -0.212 𝐞₂₄ + -0.0878 𝐞₃₄)

In [123]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(-28.2 𝐞₁₂₃ + 44.3 𝐞₁₂₄ + 23.0 𝐞₁₃₄ + -13.0 𝐞₂₃₄,
 -28.2 𝐞₁₂₃ + 44.3 𝐞₁₂₄ + 23.0 𝐞₁₃₄ + -13.0 𝐞₂₃₄)

In [124]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.547 + 2.49 𝐞₁ + 3.83 𝐞₂ + 1.32 𝐞₃ + 1.39 𝐞₄ + -3.56 𝐞₁₂ + -0.894 𝐞₁₃ + -7.24 𝐞₁₄ + -15.8 𝐞₂₃ + 7.21 𝐞₂₄ + -0.231 𝐞₃₄ + 1.43e+02 𝐞₁₂₃ + -73.5 𝐞₁₂₄ + -9.07 𝐞₁₃₄ + 32.7 𝐞₂₃₄ + -50.5 𝐞₁₂₃₄,
 0.547 + 2.49 𝐞₁ + 3.83 𝐞₂ + 1.32 𝐞₃ + 1.39 𝐞₄ + -3.56 𝐞₁₂ + -0.894 𝐞₁₃ + -7.24 𝐞₁₄ + -15.8 𝐞₂₃ + 7.21 𝐞₂₄ + -0.231 𝐞₃₄ + 1.43e+02 𝐞₁₂₃ + -73.5 𝐞₁₂₄ + -9.07 𝐞₁₃₄ + 32.7 𝐞₂₃₄ + -50.5 𝐞₁₂₃₄)

In [125]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.416 + 4.01 𝐞₁ + 3.2 𝐞₂ + 2.85 𝐞₃ + 0.0516 𝐞₄ + -18.9 𝐞₁₂ + 3.23 𝐞₁₃ + -3.2 𝐞₁₄ + 10.9 𝐞₂₃ + -1.4 𝐞₂₄ + -0.461 𝐞₃₄ + -63.7 𝐞₁₂₃ + 59.6 𝐞₁₂₄ + 4.52 𝐞₁₃₄ + -13.1 𝐞₂₃₄ + -2.15e+02 𝐞₁₂₃₄,
 0.416 + 4.01 𝐞₁ + 3.2 𝐞₂ + 2.85 𝐞₃ + 0.0516 𝐞₄ + -18.9 𝐞₁₂ + 3.23 𝐞₁₃ + -3.2 𝐞₁₄ + 10.9 𝐞₂₃ + -1.4 𝐞₂₄ + -0.461 𝐞₃₄ + -63.7 𝐞₁₂₃ + 59.6 𝐞₁₂₄ + 4.52 𝐞₁₃₄ + -13.1 𝐞₂₃₄ + -2.15e+02 𝐞₁₂₃₄)

In [126]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.416 + 4.01 𝐞₁ + 3.2 𝐞₂ + 18.9 𝐞₁₂, 0.416 + 4.01 𝐞₁ + 3.2 𝐞₂ + 18.9 𝐞₁₂)

In [127]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [128]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.416 + 0.951 𝐞₁ + 0.2 𝐞₂ + 5.32 𝐞₃ + 2.67 𝐞₄ + -0.267 𝐞₁₂ + -0.868 𝐞₁₃ + -0.304 𝐞₁₄ + 1.31 𝐞₂₃ + 0.686 𝐞₂₄ + 0.741 𝐞₃₄,
 0.416 + 0.951 𝐞₁ + 0.2 𝐞₂ + 5.32 𝐞₃ + 2.67 𝐞₄ + -0.267 𝐞₁₂ + -0.868 𝐞₁₃ + -0.304 𝐞₁₄ + 1.31 𝐞₂₃ + 0.686 𝐞₂₄ + 0.741 𝐞₃₄)

In [129]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.204 + 2.02 𝐞₁ + 1.25 𝐞₂ + 4.52 𝐞₃ + 2.14 𝐞₄ + 4.55 𝐞₁₂ + -1.86 𝐞₁₃ + -11.5 𝐞₁₄ + -7.26 𝐞₂₃ + -6.19 𝐞₂₄ + -9.11 𝐞₃₄ + 35.3 𝐞₁₂₃ + 26.6 𝐞₁₂₄ + 1.24e+02 𝐞₁₃₄ + 0.423 𝐞₂₃₄ + 1.21e+02 𝐞₁₂₃₄,
 0.204 + 2.02 𝐞₁ + 1.25 𝐞₂ + 4.52 𝐞₃ + 2.14 𝐞₄ + 4.55 𝐞₁₂ + -1.86 𝐞₁₃ + -11.5 𝐞₁₄ + -7.26 𝐞₂₃ + -6.19 𝐞₂₄ + -9.11 𝐞₃₄ + 35.3 𝐞₁₂₃ + 26.6 𝐞₁₂₄ + 1.24e+02 𝐞₁₃₄ + 0.423 𝐞₂₃₄ + 1.21e+02 𝐞₁₂₃₄)

In [130]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(-49.7 𝐞₁ + 24.7 𝐞₂ + 22.8 𝐞₃ + -31.4 𝐞₄,
 7.25 𝐞₁ + -1.2 𝐞₂ + -11.5 𝐞₃ + -3.11 𝐞₄,
 -49.7 𝐞₁ + 24.7 𝐞₂ + 22.8 𝐞₃ + -31.4 𝐞₄)

In [131]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(-24.4 𝐞₁ + 38.8 𝐞₂ + -27.8 𝐞₃ + 23.8 𝐞₄,
 -4.22 𝐞₁ + -12.3 𝐞₂ + 4.62 𝐞₃ + -0.386 𝐞₄,
 -24.4 𝐞₁ + 38.8 𝐞₂ + -27.8 𝐞₃ + 23.8 𝐞₄)

In [132]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

6.99 6.99
-0.793 -0.793
-22.7 -22.7


In [133]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(-49.7 𝐞₁ + 24.7 𝐞₂ + 22.8 𝐞₃ + -31.4 𝐞₄,
 -49.7 𝐞₁ + 24.7 𝐞₂ + 22.8 𝐞₃ + -31.4 𝐞₄)

In [134]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, -2.49e+02 𝐞₁₂₃₄, -2.49e+02 𝐞₁₂₃₄)

In [135]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(-2.49e+02, -2.49e+02, -2.49e+02)

In [136]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-6.65 𝐞₁₃, -6.65 𝐞₁₃, -6.65 𝐞₁₃)

In [137]:
#1.17a
outermorphism(g, I, alg, h), Ip

(6.65 𝐞₂₃, 1 𝐞₂₃)

In [138]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(6.65 𝐞₁₂, 1 𝐞₁₂)

In [139]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

6.651682571651819

In [140]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.114 + 0.948 𝐞₁ + 0.565 𝐞₂ + 0.0466 𝐞₁₂,
 0.114 + 0.948 𝐞₁ + 0.565 𝐞₂ + 0.544 𝐞₃ + 0.55 𝐞₄ + 0.0466 𝐞₁₂ + 0.512 𝐞₁₃ + 0.48 𝐞₁₄ + 0.895 𝐞₂₃ + 0.0251 𝐞₂₄ + 0.876 𝐞₃₄ + 0.907 𝐞₁₂₃ + 0.367 𝐞₁₂₄ + 0.931 𝐞₁₃₄ + 0.909 𝐞₂₃₄ + 0.992 𝐞₁₂₃₄)

In [141]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.959 + 0.338 𝐞₂ + 0.193 𝐞₃ + 0.78 𝐞₂₃,
 0.959 + 0.701 𝐞₁ + 0.338 𝐞₂ + 0.193 𝐞₃ + 0.84 𝐞₄ + 0.949 𝐞₁₂ + 0.787 𝐞₁₃ + 0.565 𝐞₁₄ + 0.78 𝐞₂₃ + 0.487 𝐞₂₄ + 0.98 𝐞₃₄ + 0.805 𝐞₁₂₃ + 0.162 𝐞₁₂₄ + 0.935 𝐞₁₃₄ + 0.245 𝐞₂₃₄ + 0.556 𝐞₁₂₃₄)

In [142]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(-10.1 𝐞₁₂₃ + 6.7 𝐞₁₂₄ + 3.95 𝐞₁₃₄ + 29.1 𝐞₂₃₄,
 -10.1 𝐞₁₂₃ + 6.7 𝐞₁₂₄ + 3.95 𝐞₁₃₄ + 29.1 𝐞₂₃₄,
 1.84e+02 + 6.8 𝐞₁₂ + -1.3e+02 𝐞₁₃ + -1.15e+02 𝐞₁₄ + 51.7 𝐞₂₃ + 1.31e+02 𝐞₂₄ + -1.55e+02 𝐞₃₄,
 1.84e+02 + 6.8 𝐞₁₂ + -1.3e+02 𝐞₁₃ + -1.15e+02 𝐞₁₄ + 51.7 𝐞₂₃ + 1.31e+02 𝐞₂₄ + -1.55e+02 𝐞₃₄)

In [143]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-4.69 + -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄,
 -4.69 + -2.06 𝐞₁ + 3.53 𝐞₂ + 5.67 𝐞₃ + 5.58 𝐞₄ + 2.77 𝐞₁₂ + 6.72 𝐞₁₃ + 5.36 𝐞₁₄ + -0.519 𝐞₂₃ + -1.59 𝐞₂₄ + 2.39 𝐞₃₄ + 7.26 𝐞₁₂₃ + -2.04 𝐞₁₂₄ + 2.08 𝐞₁₃₄ + 0.265 𝐞₂₃₄ + 2.16 𝐞₁₂₃₄)

In [144]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([3.53 𝐞₁ + 5.96 𝐞₂ + 3.14 𝐞₃ + -0.54 𝐞₄,
  -2.32 𝐞₁ + 9.02 𝐞₂ + 4.73 𝐞₃ + -4.73 𝐞₄,
  3.0 𝐞₁ + 3.03 𝐞₂ + 7.1 𝐞₃ + -1.81 𝐞₄,
  3.35 𝐞₁ + 5.52 𝐞₂ + 2.35 𝐞₃ + 0.493 𝐞₄],
 [3.53 𝐞₁ + 5.96 𝐞₂ + 3.14 𝐞₃ + -0.54 𝐞₄,
  -2.32 𝐞₁ + 9.02 𝐞₂ + 4.73 𝐞₃ + -4.73 𝐞₄,
  3.0 𝐞₁ + 3.03 𝐞₂ + 7.1 𝐞₃ + -1.81 𝐞₄,
  3.35 𝐞₁ + 5.52 𝐞₂ + 2.35 𝐞₃ + 0.493 𝐞₄])

In [145]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-4.69, -4.69, -4.69)

In [146]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(-54.1 + 6.75 𝐞₁₂ + 24.2 𝐞₁₃ + 18.7 𝐞₁₄ + -12.1 𝐞₂₃ + 31.0 𝐞₂₄ + -12.1 𝐞₃₄ + 11.3 𝐞₁₂₃₄,
 -54.1 + 6.75 𝐞₁₂ + 24.2 𝐞₁₃ + 18.7 𝐞₁₄ + -12.1 𝐞₂₃ + 31.0 𝐞₂₄ + -12.1 𝐞₃₄ + 11.3 𝐞₁₂₃₄)

In [147]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(891.7275633934678, 891.7275633934672, 891.7275633934682)

In [148]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(4.54e+02 + 64.3 𝐞₁₂ + -1.61e+02 𝐞₁₃ + -1.67e+02 𝐞₁₄ + 61.5 𝐞₂₃ + 3.25e+02 𝐞₂₄ + -1.14e+02 𝐞₃₄,
 4.54e+02 + 64.3 𝐞₁₂ + -1.61e+02 𝐞₁₃ + -1.67e+02 𝐞₁₄ + 61.5 𝐞₂₃ + 3.25e+02 𝐞₂₄ + -1.14e+02 𝐞₃₄)

In [149]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(-19.4, -19.448452341909174)

In [150]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(0.918 𝐞₁ + -1.93 𝐞₂ + 1.1e+02 𝐞₃ + -76.0 𝐞₄,
 0.918 𝐞₁ + -1.93 𝐞₂ + 1.1e+02 𝐞₃ + -76.0 𝐞₄,
 0.76 𝐞₁ + 0.00917 𝐞₂ + 0.0856 𝐞₃ + 0.0382 𝐞₄,
 0.76 𝐞₁ + 0.00917 𝐞₂ + 0.0856 𝐞₃ + 0.0382 𝐞₄)

In [151]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

5.68e-14 𝐞₁ + 1.14e-13 𝐞₃

In [152]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([-6.79761807+1.56863125j, -6.79761807-1.56863125j,
         3.22196665+0.j        ,  5.68677084+0.j        ]),
 array([-6.79761807+1.56863125j, -6.79761807-1.56863125j,
         5.68677084+0.j        ,  3.22196665+0.j        ]))

In [153]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.99566236, 0.99566236, 0.99566236, 0.99566236, 0.99566236,
       0.99566236, 0.99566236, 0.99566236])

In [154]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(0.957 𝐞₁ + 0.501 𝐞₂ + 2.68 𝐞₃ + -2.44 𝐞₄,
 0.957 𝐞₁ + 0.501 𝐞₂ + 2.68 𝐞₃ + -2.44 𝐞₄)

In [155]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(-54.12558848265338, -54.12558848265338)

In [156]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(184.47247255492016, (184.47247255492016+0j))

In [157]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(-310.4919095342065, (-310.4919095342067+0j))

In [158]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(8.92e+02 𝐞₁₂₃₄, 8.92e+02 𝐞₁₂₃₄, 8.92e+02 𝐞₁₂₃₄)

In [159]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(2.97e+02 𝐞₁₂₃₄, 2.97e+02 𝐞₁₂₃₄)

In [160]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(9.68 𝐞₁₂, 9.68 𝐞₁₂)

In [161]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-48.09785564173963, -48.09785564173963)

In [162]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-1.04 𝐞₁ + 2.08 𝐞₂, -1.04 𝐞₁ + 1.41 𝐞₂)

In [163]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(-3.24 𝐞₃, -3.24 𝐞₃)

In [164]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(3.24 𝐞₁₂, 3.24 𝐞₁₂)

In [165]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([2, 3], [(0, 1)]),
 array([-6.79761807+1.56863125j, -6.79761807-1.56863125j,
         3.22196665+0.j        ,  5.68677084+0.j        ]))

In [166]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[3.22196665 3.22196665 3.22196665 3.22196665] [  4.05318735 -19.73178326  -4.37204327   6.94782396]
[5.68677084 5.68677084 5.68677084 5.68677084] [ 4.39002943  9.14715106  5.41454224 -9.09437198]
[48.66821549 48.66821549 48.66821549 48.66821549 48.66821549 48.66821549] [  46.14560027   37.04789562 -307.73188877   35.70858836   42.23994445
   93.55362039]


In [167]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[ 5.55428776 21.44532212  1.76186289 -0.41471015] [3.22196665 3.22196665 3.22196665 3.22196665]
[ 7.8187381   9.52919601  4.83573732 -0.85388133] [5.68677084 5.68677084 5.68677084 5.68677084]
[ 85.41552258  88.35230247 -35.9408487  108.37154484  81.19309006
  29.05338939] [48.66821549 48.66821549 48.66821549 48.66821549 48.66821549 48.66821549]


In [168]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, , 1.0]

In [169]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(68.6 𝐞₁₂₃, 68.6 𝐞₁₂₃), (-3.64 𝐞₄, -3.64 𝐞₄)]

In [170]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[-4.17966682 -4.17966682 -4.17966682] [  4.39877569  -5.85136846 -13.96989011]
[-2.77531212 -2.77531212 -2.77531212] [  2.84374444  -6.29751613 -20.990477  ]
[5.91046291 5.91046291 5.91046291] [4.32611903 7.9196676  5.95944937]
[] []
[] []
[] []
[] []
[-3.64198263] [-3.64198263]


In [171]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[ 1.36066317,  1.95233086,  2.18726487, -4.05956457],
        [ 1.95233086, -1.01392972,  5.33307706,  3.70576847],
        [ 2.18726487,  5.33307706, -1.39124947,  1.47555536],
        [-4.05956457,  3.70576847,  1.47555536, -3.64198263]]),
 1.08 𝐞₁ + 2.07 𝐞₂ + 1.65 𝐞₃ + -3.07 𝐞₄,
 1.08 𝐞₁ + 2.07 𝐞₂ + 1.65 𝐞₃ + -3.07 𝐞₄)

In [172]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(-4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄,
 -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄)

In [173]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[ 0.00000000e+00,  2.30932179e+00, -1.31445753e+00,
         -9.52877658e-01],
        [-2.30932179e+00,  1.11022302e-16,  1.52689653e+00,
          1.38729160e+00],
        [ 1.31445753e+00, -1.52689653e+00,  0.00000000e+00,
          1.06539271e+00],
        [ 9.52877658e-01, -1.38729160e+00, -1.06539271e+00,
         -4.44089210e-16]]),
 -0.128 𝐞₁ + -1.57 𝐞₂ + 1.03 𝐞₃ + 0.621 𝐞₄,
 -0.128 𝐞₁ + -1.57 𝐞₂ + 1.03 𝐞₃ + 0.621 𝐞₄)

In [174]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(1.31,
 1.31,
 -2.33e-10 𝐞₁₂ + 1.16e-10 𝐞₁₃ + -1.16e-10 𝐞₁₄ + 1.16e-10 𝐞₂₃ + -1.16e-10 𝐞₂₄ + 3.49e-10 𝐞₃₄)

In [175]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.08 𝐞₁ + 2.07 𝐞₂ + 1.65 𝐞₃ + -3.07 𝐞₄,
 1.08 𝐞₁ + 2.07 𝐞₂ + 1.65 𝐞₃ + -3.07 𝐞₄)

In [176]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-2.32e+05 𝐞₁ + -2.83e-12 𝐞₂ + 3.82e-11 𝐞₃ + -3.77e-12 𝐞₄,
 7.97e-14 𝐞₁ + -6.53e+03 𝐞₂ + -1.75e-13 𝐞₃ + 2.83e-13 𝐞₄,
 -2.61e-14 𝐞₁ + 4.24e-15 𝐞₂ + -1.58e+02 𝐞₃ + -1.37e-14 𝐞₄,
 8.33e-17 𝐞₁ + -2.22e-16 𝐞₂ + 4.44e-16 𝐞₃ + -5.12 𝐞₄]

In [177]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [178]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -4.69 + -8.88e-16 𝐞₁₂ + 4.44e-16 𝐞₁₃ + -8.88e-16 𝐞₂₃,
 -67.5 + 4.44e-15 𝐞₁₃ + 2.66e-15 𝐞₁₄ + 7.11e-15 𝐞₂₄ + -7.11e-15 𝐞₃₄,
 1.73e+02 + 1.6e-14 𝐞₁₂ + -1.42e-14 𝐞₁₃ + 2.84e-14 𝐞₁₄ + -2.84e-14 𝐞₂₃ + 1.42e-14 𝐞₂₄ + -4.26e-14 𝐞₃₄,
 1.1e+03]

In [179]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([-8.42137908, -5.96475127,  6.15264437,  3.54698731]),
 array([-8.42137908, -5.96475127,  3.54698731,  6.15264437]))

In [180]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.37 𝐞₁ + -0.577 𝐞₂ + 0.174 𝐞₃ + 0.708 𝐞₄,
  -0.319 𝐞₁ + -0.424 𝐞₂ + 0.765 𝐞₃ + -0.367 𝐞₄,
  -0.776 𝐞₁ + 0.237 𝐞₂ + 0.0854 𝐞₃ + 0.578 𝐞₄,
  -0.399 𝐞₁ + -0.657 𝐞₂ + -0.615 𝐞₃ + -0.176 𝐞₄],
 [-0.37 𝐞₁ + 0.577 𝐞₂ + -0.174 𝐞₃ + -0.708 𝐞₄,
  -0.319 𝐞₁ + -0.424 𝐞₂ + 0.765 𝐞₃ + -0.367 𝐞₄,
  0.776 𝐞₁ + -0.237 𝐞₂ + -0.0854 𝐞₃ + -0.578 𝐞₄,
  0.399 𝐞₁ + 0.657 𝐞₂ + 0.615 𝐞₃ + 0.176 𝐞₄])

In [181]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-8.42137908 -8.42137908 -8.42137908 -8.42137908]
[70.91962557 70.91962557 70.91962557 70.91962557 70.91962557 70.91962557]


In [182]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([0.34 𝐞₁₂ + -0.338 𝐞₁₃ + -0.0898 𝐞₁₄ + 0.367 𝐞₂₃ + -0.511 𝐞₂₄ + 0.605 𝐞₃₄,
  -0.776 𝐞₁ + 0.237 𝐞₂ + 0.0854 𝐞₃ + 0.578 𝐞₄,
  -0.399 𝐞₁ + -0.657 𝐞₂ + -0.615 𝐞₃ + -0.176 𝐞₄],
 [0.34 𝐞₁₂ + -0.338 𝐞₁₃ + -0.0898 𝐞₁₄ + 0.367 𝐞₂₃ + -0.511 𝐞₂₄ + 0.605 𝐞₃₄,
  0.776 𝐞₁ + -0.237 𝐞₂ + -0.0854 𝐞₃ + -0.578 𝐞₄,
  0.399 𝐞₁ + 0.657 𝐞₂ + 0.615 𝐞₃ + 0.176 𝐞₄])

In [183]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[ 0.174254  ,  0.1628331 , -0.11664313, -0.96411374],
        [ 0.1628331 , -0.97742003, -0.01617483, -0.13369308],
        [-0.11664313, -0.01617483, -0.98841339,  0.0957691 ],
        [-0.96411374, -0.13369308,  0.0957691 , -0.20842058]]),
 [0.9999999999999999, 0.0, -1.0408340855860843e-17, 0.9999999999999999],
 array([ 1., -1., -1., -1.]))

In [184]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(0.0872 𝐞₁ + 0.108 𝐞₂ + -0.17 𝐞₃ + -0.734 𝐞₄,
 0.0872 𝐞₁ + 0.108 𝐞₂ + -0.17 𝐞₃ + -0.734 𝐞₄)

In [185]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([-0.526 𝐞₁ + 0.761 𝐞₂ + -0.379 𝐞₃,
  0.678 𝐞₁ + -0.702 𝐞₂ + 0.216 𝐞₃,
  0.615 𝐞₁ + 0.539 𝐞₂ + 0.576 𝐞₃,
  1.0 𝐞₄,
  1 𝐞₁₂₃],
 [4.759695322438004, 3.6307345467071306, 0.896161509372607, 0.0, 0.0])

In [186]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(-0.201, -0.201)

In [187]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(-0.128 𝐞₁ + -1.57 𝐞₂ + 1.03 𝐞₃ + 0.621 𝐞₄,
 -0.128 𝐞₁ + -1.57 𝐞₂ + 1.03 𝐞₃ + 0.621 𝐞₄,
 8.33e-17)

In [188]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(4.37e-10 + -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄,
 -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄,
 -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄,
 -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄)

In [189]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [190]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  -2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
  2.83 𝐞₁₂₃₄],
 [-3.33e-16 + -4.62 𝐞₁₂ + 2.63 𝐞₁₃ + 1.91 𝐞₁₄ + -3.05 𝐞₂₃ + -2.77 𝐞₂₄ + -2.13 𝐞₃₄,
  13.4 + 6.66e-16 𝐞₁₂ + -6.66e-16 𝐞₁₃ + 6.66e-16 𝐞₁₄ + -8.88e-16 𝐞₂₄ + 4.44e-16 𝐞₃₄ + 11.3 𝐞₁₂₃₄,
  4.44e-16 + -6.03 𝐞₁₂ + 7.85 𝐞₁₃ + -8.64 𝐞₁₄ + 5.39 𝐞₂₃ + -7.44 𝐞₂₄ + -13.1 𝐞₃₄,
  8.0])

In [191]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[-2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
 3.01 𝐞₁₂ + -3.92 𝐞₁₃ + 4.32 𝐞₁₄ + -2.7 𝐞₂₃ + 3.72 𝐞₂₄ + 6.53 𝐞₃₄]

In [192]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [193]:
x|F, fm(x), x|F|F, fm(fm(x))

(-0.128 𝐞₁ + -1.57 𝐞₂ + 1.03 𝐞₃ + 0.621 𝐞₄,
 -0.128 𝐞₁ + -1.57 𝐞₂ + 1.03 𝐞₃ + 0.621 𝐞₄,
 -5.57 𝐞₁ + 2.72 𝐞₂ + 2.89 𝐞₃ + 0.966 𝐞₄,
 -5.57 𝐞₁ + 2.72 𝐞₂ + 2.89 𝐞₃ + 0.966 𝐞₄)

In [194]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 13.359794210044496, 8.002816172752224],
 [1,
  3.3306690738754696e-16,
  13.3597942100445,
  -4.440892098500626e-16,
  8.002816172752228],
 array([-12.73119524,  -0.62859897]),
 array([3.56808005, 0.79284233]),
 array([-1.93529922e-16+3.56808005j, -1.93529922e-16-3.56808005j,
         1.11022302e-16+0.79284233j,  1.11022302e-16-0.79284233j]),
 [1,
  26.719588420089003,
  194.489733680243,
  213.83195433757115,
  64.04506669486476])

In [195]:
Ck[1], Ck[1] | Ck[2]

(-2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
 3.01 𝐞₁₂ + -3.92 𝐞₁₃ + 4.32 𝐞₁₄ + -2.7 𝐞₂₃ + 3.72 𝐞₂₄ + 6.53 𝐞₃₄)

In [196]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(-2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
 -2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
 4.44e-16 + -1.11e-16 𝐞₁₂ + 1.11e-16 𝐞₂₄ + -5.55e-17 𝐞₃₄ + 2.83 𝐞₁₂₃₄,
 4.44e-16 + 1.11e-16 𝐞₁₂ + -1.11e-16 𝐞₂₄ + 5.55e-17 𝐞₃₄ + 2.83 𝐞₁₂₃₄,
 -13.4 + -2.22e-16 𝐞₁₂ + 2.22e-16 𝐞₁₃ + 5.66 𝐞₁₂₃₄,
 -0.629 + 1.39e-17 𝐞₁₂ + -1.11e-16 𝐞₁₂₃₄,
 -12.7 + -2.22e-16 𝐞₁₂ + 1.11e-16 𝐞₁₃ + -1.78e-15 𝐞₁₂₃₄)

In [197]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-12.73119524351554, -0.6285989665289557],
 array([-12.73119524,  -0.62859897]))

In [198]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(-0.129 𝐞₁₂ + 0.256 𝐞₁₃ + -0.406 𝐞₁₄ + 0.302 𝐞₂₃ + -0.235 𝐞₂₄ + -0.484 𝐞₃₄,
 -0.129 𝐞₁₂ + 0.256 𝐞₁₃ + -0.406 𝐞₁₄ + 0.302 𝐞₂₃ + -0.235 𝐞₂₄ + -0.484 𝐞₃₄,
 -2.18 𝐞₁₂ + 1.06 𝐞₁₃ + 1.36 𝐞₁₄ + -1.83 𝐞₂₃ + -1.15 𝐞₂₄ + -0.581 𝐞₃₄,
 -2.18 𝐞₁₂ + 1.06 𝐞₁₃ + 1.36 𝐞₁₄ + -1.83 𝐞₂₃ + -1.15 𝐞₂₄ + -0.581 𝐞₃₄)

In [199]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.linalg.solve(np.array([np.ones(m), Lk]), [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [200]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-0.0866 + -8.89e-14 𝐞₁ + -2.95e-13 𝐞₂ + 1.78e-13 𝐞₃ + 1.23e-13 𝐞₄ + -4.3e-13 𝐞₁₂ + 1.05e-13 𝐞₁₃ + 8.53e-14 𝐞₁₄ + -4.64e-14 𝐞₂₃ + -1.29e-13 𝐞₂₄ + 3.12e-13 𝐞₃₄ + 5e-14 𝐞₁₂₃ + 1.3e-13 𝐞₁₂₄ + 2.24e-13 𝐞₁₃₄ + 1.57e-13 𝐞₂₃₄ + 3.66e-14 𝐞₁₂₃₄,
 [array([-5.59305806e-13,  1.97302717e-13, -2.47683882e-12, -1.78736252e-14,
          2.26889855e-13, -8.00185194e-14]),
  array([1., 1., 1., 1., 1., 1.]),
  array([-2.11468084e-13, -2.11631181e-13, -2.11498460e-13, -2.09316364e-13,
         -2.11528305e-13, -2.11638663e-13]),
  array([ 0.37809027, -1.07262173,  0.08539048, 11.71090712, -0.93229512,
          2.64487102])])

In [201]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

-1.42e-13 𝐞₁₂ + -2.84e-14 𝐞₁₃ + 1.42e-14 𝐞₁₄ + 1.14e-13 𝐞₂₃ + 2.84e-14 𝐞₂₄ + -5.68e-14 𝐞₃₄

In [202]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.247 𝐞₁ + -0.173 𝐞₂ + -0.236 𝐞₃ + -0.0937 𝐞₄,
 -0.247 𝐞₁ + -0.173 𝐞₂ + -0.236 𝐞₃ + -0.0937 𝐞₄ + -1.13e-17 𝐞₁₂₃ + 4.37e-17 𝐞₁₂₄ + -3.05e-17 𝐞₁₃₄ + 4.37e-17 𝐞₂₃₄)

In [203]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(-0.129 𝐞₁₂ + 0.256 𝐞₁₃ + -0.406 𝐞₁₄ + 0.302 𝐞₂₃ + -0.235 𝐞₂₄ + -0.484 𝐞₃₄,
 -0.129 𝐞₁₂ + 0.256 𝐞₁₃ + -0.406 𝐞₁₄ + 0.302 𝐞₂₃ + -0.235 𝐞₂₄ + -0.484 𝐞₃₄)

In [204]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [205]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [206]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(1.25 𝐞₁₂ + -1.83 𝐞₁₃ + -2.93 𝐞₁₄ + 2.1 𝐞₂₃ + 4.63 𝐞₂₄ + 0.401 𝐞₃₄,
 1.25 𝐞₁₂ + -1.83 𝐞₁₃ + -2.93 𝐞₁₄ + 2.1 𝐞₂₃ + 4.63 𝐞₂₄ + 0.401 𝐞₃₄)

In [207]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(1.25 𝐞₁₂ + -1.83 𝐞₁₃ + -2.93 𝐞₁₄ + 2.1 𝐞₂₃ + 4.63 𝐞₂₄ + 0.401 𝐞₃₄,
 1.25 𝐞₁₂ + -1.83 𝐞₁₃ + -2.93 𝐞₁₄ + 2.1 𝐞₂₃ + 4.63 𝐞₂₄ + 0.401 𝐞₃₄)

In [208]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(-0.0811 𝐞₁₂ + 0.161 𝐞₁₃ + -0.255 𝐞₁₄ + 0.19 𝐞₂₃ + -0.148 𝐞₂₄ + -0.305 𝐞₃₄,
 -0.0811 𝐞₁₂ + 0.161 𝐞₁₃ + -0.255 𝐞₁₄ + 0.19 𝐞₂₃ + -0.148 𝐞₂₄ + -0.305 𝐞₃₄)

In [209]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(-3.52 𝐞₁ + 9.12 𝐞₂ + -9.18 𝐞₃ + -6.05 𝐞₄,
 5.54 𝐞₁ + -4.24 𝐞₂ + -5.01 𝐞₃ + -3.6 𝐞₄,
 18.1 𝐞₁ + 8.08 𝐞₂ + -0.454 𝐞₃ + 13.5 𝐞₄,
 36.2 𝐞₁ + -18.6 𝐞₂ + 7.89 𝐞₃ + 18.4 𝐞₄)

In [210]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(0.526 𝐞₁ + 0.218 𝐞₂ + -0.133 𝐞₃ + 0.376 𝐞₄,
 0.526 𝐞₁ + 0.218 𝐞₂ + -0.133 𝐞₃ + 0.376 𝐞₄)

In [211]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(0.162 𝐞₁ + 0.547 𝐞₂ + -0.283 𝐞₃ + 0.0127 𝐞₄,
 2.41 𝐞₁ + 0.542 𝐞₂ + -0.723 𝐞₃ + -1.25 𝐞₄ + -1.32e-16 𝐞₁₂₃ + -1.06e-16 𝐞₁₂₄ + 1.08e-17 𝐞₁₃₄ + -2.48e-17 𝐞₂₃₄)

In [212]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(-2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
 -2.31 𝐞₁₂ + 1.31 𝐞₁₃ + 0.953 𝐞₁₄ + -1.53 𝐞₂₃ + -1.39 𝐞₂₄ + -1.07 𝐞₃₄,
 -0.102 𝐞₁₂ + 0.203 𝐞₁₃ + -0.322 𝐞₁₄ + 0.239 𝐞₂₃ + -0.186 𝐞₂₄ + -0.384 𝐞₃₄,
 -0.102 𝐞₁₂ + 0.203 𝐞₁₃ + -0.322 𝐞₁₄ + 0.239 𝐞₂₃ + -0.186 𝐞₂₄ + -0.384 𝐞₃₄)

In [213]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([117.51579546, -60.38952098, -18.05561243, -41.92509762,
       -45.29080415,  -9.76478956])

In [214]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.37611757, 0.37611757, 0.37611757, 0.37611757, 0.37611757,
        0.37611757])]

In [215]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([0.6132842458166727], array([0.37611757]), [0.3761175661489226])

In [216]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.438 𝐞₁ + 0.281 𝐞₂ + -0.307 𝐞₃ + -0.919 𝐞₄],
 [-0.438 𝐞₁ + 0.281 𝐞₂ + -0.307 𝐞₃ + -0.919 𝐞₄ + -1.39e-17 𝐞₁₂₃ + -5.55e-17 𝐞₁₂₄ + 1.39e-17 𝐞₂₃₄],
 [-0.438 𝐞₁ + 0.281 𝐞₂ + -0.307 𝐞₃ + -0.919 𝐞₄ + 1.39e-17 𝐞₁₂₃ + 5.55e-17 𝐞₁₂₄ + -6.94e-18 𝐞₁₃₄ + -6.94e-18 𝐞₂₃₄])

In [217]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.691 𝐞₁ + 0.763 𝐞₂ + 0.0537 𝐞₃ + 0.384 𝐞₄],
 [0.691 𝐞₁ + 0.763 𝐞₂ + 0.0537 𝐞₃ + 0.384 𝐞₄ + -1.39e-17 𝐞₁₂₃ + -5.55e-17 𝐞₁₂₄ + 6.94e-18 𝐞₁₃₄ + 6.94e-18 𝐞₂₃₄],
 [0.691 𝐞₁ + 0.763 𝐞₂ + 0.0537 𝐞₃ + 0.384 𝐞₄ + 1.39e-17 𝐞₁₂₃ + 5.55e-17 𝐞₁₂₄ + -1.39e-17 𝐞₂₃₄])

In [218]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.1000000000444248], [1.1000000000444248])

In [219]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([0.613 + 0.431 𝐞₁₂ + -0.154 𝐞₁₃ + -0.381 𝐞₁₄ + -0.203 𝐞₂₃ + -0.66 𝐞₂₄ + 0.0557 𝐞₃₄],
 [0.613 + 0.431 𝐞₁₂ + -0.154 𝐞₁₃ + -0.381 𝐞₁₄ + -0.203 𝐞₂₃ + -0.66 𝐞₂₄ + 0.0557 𝐞₃₄])

In [220]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.6132842458166725

In [221]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 1.42123923, -0.32123923])

In [222]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(0.162 𝐞₁ + 0.547 𝐞₂ + -0.283 𝐞₃ + 0.0127 𝐞₄,
 0.162 𝐞₁ + 0.547 𝐞₂ + -0.283 𝐞₃ + 0.0127 𝐞₄ + 5.2e-18 𝐞₁₂₃ + 2.08e-17 𝐞₁₂₄ + 6.94e-18 𝐞₁₃₄)

In [223]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-0.32123923,  0.61328425,  0.61328425,  1.42123923]),
 [0.71 𝐞₁ + -0.389 𝐞₂ + 0.239 𝐞₃ + -0.536 𝐞₄,
  -0.179 𝐞₁ + 0.554 𝐞₂ + -0.282 𝐞₃ + -0.763 𝐞₄,
  -0.627 𝐞₁ + -0.698 𝐞₂ + -0.0465 𝐞₃ + -0.343 𝐞₄,
  -0.268 𝐞₁ + 0.233 𝐞₂ + 0.928 𝐞₃ + -0.111 𝐞₄])

In [224]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([0.10319464, 0.10319464, 0.10319464, 0.10319464]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([2.01992094, 2.01992094, 2.01992094, 2.01992094])]

In [225]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(0.162 𝐞₁ + 0.547 𝐞₂ + -0.283 𝐞₃ + 0.0127 𝐞₄,
 0.162 𝐞₁ + 0.547 𝐞₂ + -0.283 𝐞₃ + 0.0127 𝐞₄ + -3.68e-18 𝐞₁₂₃ + -4.62e-17 𝐞₁₂₄ + 5.66e-19 𝐞₁₃₄ + 2.12e-18 𝐞₂₃₄)

In [226]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.416 + 0.761 𝐞₁ + 0.917 𝐞₂ + 0.786 𝐞₃ + 0.334 𝐞₄ + 0.0626 𝐞₁₂ + 0.462 𝐞₁₃ + 0.418 𝐞₁₄ + 0.913 𝐞₂₃ + 0.653 𝐞₂₄ + 0.109 𝐞₃₄ + 0.146 𝐞₁₂₃ + 0.175 𝐞₁₂₄ + 0.14 𝐞₁₃₄ + 0.612 𝐞₂₃₄ + 0.865 𝐞₁₂₃₄,
 0.416 + 0.618 𝐞₁ + 1.32 𝐞₂ + 1.43 𝐞₃ + 0.455 𝐞₄ + 0.149 𝐞₁₂ + 1.01 𝐞₁₃ + 0.372 𝐞₁₄ + 2.16 𝐞₂₃ + 0.807 𝐞₂₄ + 0.244 𝐞₃₄ + 0.921 𝐞₁₂₃ + 0.439 𝐞₁₂₄ + 0.677 𝐞₁₃₄ + 1.32 𝐞₂₃₄ + 0.264 𝐞₁₂₃₄)

In [227]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.416 + 0.761 𝐞₁ + 0.917 𝐞₂ + 0.786 𝐞₃ + 0.334 𝐞₄ + 0.0626 𝐞₁₂ + 0.462 𝐞₁₃ + 0.418 𝐞₁₄ + 0.913 𝐞₂₃ + 0.653 𝐞₂₄ + 0.109 𝐞₃₄ + 0.146 𝐞₁₂₃ + 0.175 𝐞₁₂₄ + 0.14 𝐞₁₃₄ + 0.612 𝐞₂₃₄ + 0.865 𝐞₁₂₃₄,
 0.416 + 0.761 𝐞₁ + 0.917 𝐞₂ + 0.786 𝐞₃ + 0.334 𝐞₄ + 0.0626 𝐞₁₂ + 0.462 𝐞₁₃ + 0.418 𝐞₁₄ + 0.913 𝐞₂₃ + 0.653 𝐞₂₄ + 0.109 𝐞₃₄ + 0.146 𝐞₁₂₃ + 0.175 𝐞₁₂₄ + 0.14 𝐞₁₃₄ + 0.612 𝐞₂₃₄ + 0.865 𝐞₁₂₃₄)

In [228]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(-0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄ + 4.24e-18 𝐞₁₂₃ + 2.09e-17 𝐞₁₂₄ + 1.09e-17 𝐞₁₃₄ + -7.46e-18 𝐞₂₃₄,
 -0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄ + -4.68e-18 𝐞₁₂₃ + 2.17e-17 𝐞₁₂₄ + -2.93e-18 𝐞₁₃₄ + 3.03e-18 𝐞₂₃₄)

In [229]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([2.22e-16 + 0.323 𝐞₁₂ + -0.157 𝐞₁₃ + -0.637 𝐞₁₄ + -0.0224 𝐞₂₃ + 0.594 𝐞₂₄ + -0.333 𝐞₃₄,
  1.67e-16 + 0.472 𝐞₁₂ + -0.168 𝐞₁₃ + -0.417 𝐞₁₄ + -0.222 𝐞₂₃ + -0.723 𝐞₂₄ + 0.061 𝐞₃₄,
  -2.78e-17 + -0.333 𝐞₁₂ + -0.594 𝐞₁₃ + -0.0224 𝐞₁₄ + -0.637 𝐞₂₃ + 0.157 𝐞₂₄ + 0.323 𝐞₃₄],
 [-0.409 𝐞₁₂ + 0.225 𝐞₁₃ + -0.859 𝐞₁₄ + 0.0123 𝐞₂₃ + 0.171 𝐞₂₄ + -0.12 𝐞₃₄,
  0.472 𝐞₁₂ + -0.168 𝐞₁₃ + -0.417 𝐞₁₄ + -0.222 𝐞₂₃ + -0.723 𝐞₂₄ + 0.061 𝐞₃₄,
  0.472 𝐞₁₂ + -0.168 𝐞₁₃ + -0.417 𝐞₁₄ + -0.222 𝐞₂₃ + -0.723 𝐞₂₄ + 0.061 𝐞₃₄,
  0.274 𝐞₁₂ + 0.477 𝐞₁₃ + 0.142 𝐞₁₄ + 0.477 𝐞₂₃ + -0.221 𝐞₂₄ + -0.633 𝐞₃₄])

In [230]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([0.71 𝐞₁ + -0.389 𝐞₂ + 0.239 𝐞₃ + -0.536 𝐞₄,
  -0.179 𝐞₁ + 0.554 𝐞₂ + -0.282 𝐞₃ + -0.763 𝐞₄,
  -0.627 𝐞₁ + -0.698 𝐞₂ + -0.0465 𝐞₃ + -0.343 𝐞₄,
  -0.268 𝐞₁ + 0.233 𝐞₂ + 0.928 𝐞₃ + -0.111 𝐞₄],
 [[0.71 𝐞₁ + -0.389 𝐞₂ + 0.239 𝐞₃ + -0.536 𝐞₄,
   0.71 𝐞₁ + -0.389 𝐞₂ + 0.239 𝐞₃ + -0.536 𝐞₄,
   0.71 𝐞₁ + -0.389 𝐞₂ + 0.239 𝐞₃ + -0.536 𝐞₄],
  [-0.62 𝐞₁ + -0.271 𝐞₂ + -0.196 𝐞₃ + -0.71 𝐞₄,
   -0.62 𝐞₁ + -0.271 𝐞₂ + -0.196 𝐞₃ + -0.71 𝐞₄,
   -0.179 𝐞₁ + 0.554 𝐞₂ + -0.282 𝐞₃ + -0.763 𝐞₄],
  [-0.201 𝐞₁ + -0.849 𝐞₂ + 0.208 𝐞₃ + 0.442 𝐞₄,
   -0.201 𝐞₁ + -0.849 𝐞₂ + 0.208 𝐞₃ + 0.442 𝐞₄,
   -0.627 𝐞₁ + -0.698 𝐞₂ + -0.0465 𝐞₃ + -0.343 𝐞₄],
  [-0.268 𝐞₁ + 0.233 𝐞₂ + 0.928 𝐞₃ + -0.111 𝐞₄,
   -0.268 𝐞₁ + 0.233 𝐞₂ + 0.928 𝐞₃ + -0.111 𝐞₄,
   -0.268 𝐞₁ + 0.233 𝐞₂ + 0.928 𝐞₃ + -0.111 𝐞₄]])

In [231]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(-0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄ + 4.24e-18 𝐞₁₂₃ + 2.09e-17 𝐞₁₂₄ + 1.09e-17 𝐞₁₃₄ + -7.46e-18 𝐞₂₃₄,
 -0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄)

In [232]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(-0.639 + 0.283 𝐞₁₂ + -0.296 𝐞₁₃ + 0.222 𝐞₁₄ + -0.0982 𝐞₂₃ + 0.286 𝐞₂₄ + 0.444 𝐞₃₄ + -0.295 𝐞₁₂₃₄,
 -0.639 + 0.283 𝐞₁₂ + -0.296 𝐞₁₃ + 0.222 𝐞₁₄ + -0.0982 𝐞₂₃ + 0.286 𝐞₂₄ + 0.444 𝐞₃₄ + -0.295 𝐞₁₂₃₄)

In [233]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.702 + -0.116 𝐞₁₂ + 0.23 𝐞₁₃ + -0.365 𝐞₁₄ + 0.271 𝐞₂₃ + -0.211 𝐞₂₄ + -0.435 𝐞₃₄,
 0.702 + -0.116 𝐞₁₂ + 0.23 𝐞₁₃ + -0.365 𝐞₁₄ + 0.271 𝐞₂₃ + -0.211 𝐞₂₄ + -0.435 𝐞₃₄)

In [234]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(-0.000368 𝐞₁ + 0.426 𝐞₂ + -0.17 𝐞₃ + 0.613 𝐞₄,
 -0.000368 𝐞₁ + 0.426 𝐞₂ + -0.17 𝐞₃ + 0.613 𝐞₄)

In [235]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(-0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄ + 4.24e-18 𝐞₁₂₃ + 2.09e-17 𝐞₁₂₄ + 1.09e-17 𝐞₁₃₄ + -7.46e-18 𝐞₂₃₄,
 -0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄,
 -0.132 𝐞₁ + 0.653 𝐞₂ + -0.313 𝐞₃ + 0.211 𝐞₄)

In [236]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(-0.012 𝐞₁₂₃ + -0.517 𝐞₁₂₄ + -0.447 𝐞₁₃₄ + -0.73 𝐞₂₃₄,
 -0.012 𝐞₁₂₃ + -0.517 𝐞₁₂₄ + -0.447 𝐞₁₃₄ + -0.73 𝐞₂₃₄)

In [237]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(0.69 𝐞₁₂ + -2.0 𝐞₁₃ + 1.52 𝐞₁₄ + -1.96 𝐞₂₃ + 1.44 𝐞₂₄ + 0.124 𝐞₃₄,
 0.69 𝐞₁₂ + -2.0 𝐞₁₃ + 1.52 𝐞₁₄ + -1.96 𝐞₂₃ + 1.44 𝐞₂₄ + 0.124 𝐞₃₄)

In [238]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(-1.02 𝐞₁ + -1.64 𝐞₂ + 0.026 𝐞₃ + 0.458 𝐞₄ + -2.84 𝐞₁₂₃ + -0.428 𝐞₁₂₄ + -0.412 𝐞₁₃₄ + 0.602 𝐞₂₃₄,
 -1.02 𝐞₁ + -1.64 𝐞₂ + 0.026 𝐞₃ + 0.458 𝐞₄ + -2.84 𝐞₁₂₃ + -0.428 𝐞₁₂₄ + -0.412 𝐞₁₃₄ + 0.602 𝐞₂₃₄)

In [280]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = create_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-4.65 + -5.55e-17 𝐞₁₅ + 0.125 𝐞₁₂₃₄ + -0.392 𝐞₁₂₃₅ + 0.28 𝐞₁₂₄₅ + -1.78 𝐞₁₃₄₅ + -3.88 𝐞₂₃₄₅

In [343]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.0258 𝐞₁ + 0.000859 𝐞₂ + 0.16 𝐞₃ + 0.458 𝐞₄ + 0.222 𝐞₅,
 0.0258 𝐞₁ + 0.000859 𝐞₂ + 0.16 𝐞₃ + 0.458 𝐞₄ + 0.222 𝐞₅)

In [303]:
skew_blades = bivector_split(ortho_curl/2, alg)

In [345]:
rotors = []
for blade in skew_blades:
    blade_norm = norm(blade)
    half_blade_ortho = (np.arcsin(blade_norm)/blade_norm/2) * blade.reverse()
    rotors.append(blade_exp(half_blade_ortho))
rotor = reduce(alg.gp, rotors)

In [346]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part
sym_vec = (wedge(skew_blades) | alg.pseudoscalar((1,))).normalized()
ortho(sym_vec), sym_vec

(-0.903 𝐞₁ + 0.414 𝐞₂ + 0.0652 𝐞₃ + 0.0913 𝐞₄ + 0.0292 𝐞₅,
 -0.903 𝐞₁ + 0.414 𝐞₂ + 0.0652 𝐞₃ + 0.0913 𝐞₄ + 0.0292 𝐞₅)

In [360]:
rotor.sw(sym_vec), -sym_vec.sw(sym_vec), sym_vec

(-0.903 𝐞₁ + 0.414 𝐞₂ + 0.0652 𝐞₃ + 0.0913 𝐞₄ + 0.0292 𝐞₅ + 3.75e-16 𝐞₁₂₃₄₅,
 0.903 𝐞₁ + -0.414 𝐞₂ + -0.0652 𝐞₃ + -0.0913 𝐞₄ + -0.0292 𝐞₅,
 -0.903 𝐞₁ + 0.414 𝐞₂ + 0.0652 𝐞₃ + 0.0913 𝐞₄ + 0.0292 𝐞₅)

In [347]:
outermorphism(ortho, skew_blades[0] + 2* skew_blades[1], alg), rotor.sw(skew_blades[0] + 2* skew_blades[1])

(-0.144 𝐞₁₂ + 0.0554 𝐞₁₃ + 0.456 𝐞₁₄ + 0.497 𝐞₁₅ + 0.0699 𝐞₂₃ + 1.01 𝐞₂₄ + 1.14 𝐞₂₅ + -0.385 𝐞₃₄ + 0.483 𝐞₃₅ + -0.616 𝐞₄₅,
 -0.144 𝐞₁₂ + 0.0554 𝐞₁₃ + 0.456 𝐞₁₄ + 0.497 𝐞₁₅ + 0.0699 𝐞₂₃ + 1.01 𝐞₂₄ + 1.14 𝐞₂₅ + -0.385 𝐞₃₄ + 0.483 𝐞₃₅ + -0.616 𝐞₄₅)

In [354]:
# Somehow the orientation of rotors get messy...
p = 0
rotation_tests = [P(x, blade) for blade in skew_blades]
for i, x in enumerate(rotation_tests):
    sw = rotors[i].sw(x)
    ans = ortho(x)
    if max_diff(sw, ans)<1e-5:
        continue
    if max_diff(sw, -ans)<1e-5:
        p+=1
        continue
    sw = rotors[i].reverse().sw(x)
    if max_diff(sw, ans)<1e-5:
        rotors[i] = rotors[i].reverse()
        continue
    if max_diff(sw, -ans)<1e-5:
        rotors[i] = rotors[i].reverse()
        p+=1
p, rotors

(1,
 [0.949 + 0.00937 𝐞₁₂ + -0.0619 𝐞₁₃ + 0.00497 𝐞₁₄ + -0.0104 𝐞₁₅ + -0.0994 𝐞₂₃ + -0.0255 𝐞₂₄ + 0.0119 𝐞₂₅ + 0.221 𝐞₃₄ + -0.189 𝐞₃₅ + -0.022 𝐞₄₅,
  0.85 + -0.0372 𝐞₁₂ + -0.0183 𝐞₁₃ + 0.137 𝐞₁₄ + 0.14 𝐞₁₅ + -0.0349 𝐞₂₃ + 0.283 𝐞₂₄ + 0.343 𝐞₂₅ + 0.0103 𝐞₃₄ + 0.0364 𝐞₃₅ + -0.193 𝐞₄₅])

In [358]:
rotor = reduce(alg.gp, rotors)
rotor2 = sym_vec * rotor
xd = P(x, sym_vec.dual())
(-1)**(p+1)*(rotor2.sw(x)), ortho(x)

(-0.16 𝐞₁ + -0.316 𝐞₂ + -0.00489 𝐞₃ + -0.0534 𝐞₄ + -0.281 𝐞₅ + -3.05e-17 𝐞₁₂₃₄₅,
 -0.16 𝐞₁ + -0.316 𝐞₂ + -0.00489 𝐞₃ + -0.0534 𝐞₄ + -0.281 𝐞₅)